<a href="https://colab.research.google.com/github/susmitha-neel/Vnet-medical-segmentation/blob/main/VNet_for_3D_image_segmentaion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import glob
import os
import nibabel as nib

import h5py

import tensorflow as tf
import keras
from keras.layers import *
from keras import activations
from keras.models import Model, model_from_json
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras.optimizers import Adam
from keras.callbacks import CSVLogger, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [2]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()


{}

In [4]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mv: cannot stat 'kaggle.json': No such file or directory


In [5]:
!kaggle datasets download -d andrewmvd/liver-tumor-segmentation

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/liver-tumor-segmentation
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
liver-tumor-segmentation.zip: Skipping, found more recently modified local copy (use --force to force download)


In [11]:
!unzip -q {'/content/liver-tumor-segmentation.zip'} -d /content/

replace /content/segmentations/segmentation-0.nii? [y]es, [n]o, [A]ll, [N]one, [r]ename: A n
n
n
n
n
n
n
n
n
n
n
n
n
n
n
n
n
n
n


In [7]:
liver_tumor_images = '/content/liver-tumor-segmentation.zip'  # Update with the correct folder name

# List all .nii.gz files in the directory
files = glob.glob(os.path.join(liver_tumor_images, '*.nii.gz'))

In [8]:
!mkdir '/content/volumes'
!mkdir '/content/liver_image_dataset/'

mkdir: cannot create directory ‘/content/volumes’: File exists
mkdir: cannot create directory ‘/content/liver_image_dataset/’: File exists


In [9]:
import shutil

In [10]:
content_files = os.listdir("/content/")
content_path = "/content/"
for folder in content_files:
  if folder[0] == 'v':
    volume_path = content_path+folder
    volume_files = os.listdir(volume_path)
    for vol_file in volume_files:
      dest_files = os.listdir("/content/volumes")
      if vol_file not in dest_files:
        shutil.move(volume_path+"/"+vol_file,"/content/volumes" )

NotADirectoryError: [Errno 20] Not a directory: '/content/val_data.h5'

In [ ]:
volume_path = "/content/volumes"
segment_path = "/content/segmentations"

In [ ]:
volume_files = os.listdir("/content/volumes")
segment_files = os.listdir("/content/segmentations")
for f in volume_files:
  dest_files = os.listdir("/content/liver_image_dataset")
  if f not in dest_files:
    shutil.move("/content/volumes/"+f,"/content/liver_image_dataset")
for f in segment_files:
  dest_files = os.listdir("/content/liver_image_dataset")
  if f not in dest_files:
    shutil.move("/content/segmentations/"+f,"/content/liver_image_dataset")

In [ ]:


import os
import re

import h5py
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import scipy.misc
import scipy.ndimage

INPUT_SIZE = 128  # Input feature width/height
INPUT_DEPTH = 64  # Input depth

""" K. Sahi, S. Jackson, E. Wiebe, G. Armstrong, S. Winters, R. Moore, et al., ”The value of liver windows
settings in the detection of small renal cell carcinomas on unenhanced computed tomography,”
Canadian Association of Radiologists Journal, vol. 65, pp. 71-76, 2014."""
MIN_HU = -160  # Min HU Value
MAX_HU = 240  # Max HU Value


def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=alphanum_key)


def hu_window(image):
    image[image < MIN_HU] = MIN_HU
    image[image > MAX_HU] = MAX_HU
    image = (image - MIN_HU) / (MAX_HU - MIN_HU)
    image = image.astype("float32")
    return image


def scale_volume(volume, img_depth=INPUT_DEPTH, img_px_size=INPUT_SIZE, hu_value=True):
    if hu_value:
        volume = hu_window(volume)

    size_scale_factor = img_px_size / volume.shape[0]
    depth_scale_factor = img_depth / volume.shape[-1]

    volume = scipy.ndimage.interpolation.rotate(volume, 90, reshape=False)

    vol_zoom = scipy.ndimage.interpolation.zoom(volume, [size_scale_factor, size_scale_factor,
                                                         depth_scale_factor], order=1)

    vol_zoom[vol_zoom < 0] = 0
    vol_zoom[vol_zoom > 1] = 1
    return vol_zoom


def scale_segmentation(segmentation, img_depth=INPUT_DEPTH, img_px_size=INPUT_SIZE):
    size_scale_factor = img_px_size / segmentation.shape[0]
    depth_scale_factor = img_depth / segmentation.shape[-1]

    segmentation = scipy.ndimage.interpolation.rotate(segmentation, 90, reshape=False)

    # Nearest neighbour is used to mantain classes discrete
    seg_zoom = scipy.ndimage.interpolation.zoom(segmentation, [size_scale_factor, size_scale_factor,
                                                               depth_scale_factor], order=0)

    return seg_zoom


def get_data(vol_dir, seg_dir,crop=False):
    volume = nib.load(vol_dir).get_fdata()
    segmentation = nib.load(seg_dir).get_fdata()

    if crop:
        aux = []
        for i in range(segmentation.shape[2]):
            if np.sum(segmentation[:, :, i]) > 0:
                aux.append(i)

        volume = volume[:, :, (np.min(aux)-1):(np.max(aux)+1)]
        segmentation = segmentation[:, :, (np.min(aux)-1):(np.max(aux)+1)]

    return volume, segmentation



def create_dataset(path,data, px_size=INPUT_SIZE, slice_count=INPUT_DEPTH, crop=False):
    """Returns dataset with shape (m, z, x, y, n)"""
    dataset = data
    files = sorted_alphanumeric(os.listdir(path))
    segmentations = []
    volumes = []
    for name in files:
        if name[0] == 's':
            segmentations.append(name)
        elif name[0] == 'v':
            volumes.append(name)

    # m = len(volumes)
    if data == 'train':
      m = 0
      n = 2
      r = 2
    elif data == 'val':
      m = 5
      n = 6
      r = 1
    elif data == 'test':
      m = 6
      n = 7
      r = 1
    if crop:
        print("Creating Cropped Data Set:")
    else:
        print("Creating Data Set:")

    slices = []
    print("0/%i (0%%)" % r)
    for i in range(m,n):
        volume, segmentation = get_data(path + volumes[i], path + segmentations[i], crop)

        if volume.shape[0] == 0 or volume.shape[1] == 0 or volume.shape[2] == 0:
          continue
        else:
          volume = scale_volume(volume, slice_count, px_size)
        if segmentation.shape[0] == 0 or segmentation.shape[1] == 0 or segmentation.shape[2] == 0:
          continue
        else:
          segmentation = scale_segmentation(segmentation, slice_count, px_size)
        slices.append([volume, segmentation])
        print("%i/%i (%i%%)" % (i-m+1, r, ((i-m+1)/r*100)))

    dataset = np.array(slices)

    print("Dataset finished with shape:")
    print(dataset.shape)

    return dataset


def write_dataset(data_set, path):
    h5f = h5py.File(path, 'w')
    h5f.create_dataset('data', data=np.expand_dims(data_set[:, 0, :, :, :], -1))
    truth = np.expand_dims(data_set[:, 1, :, :, :], -1)
    truth = divide_segmentation(truth)
    h5f.create_dataset('truth', data=truth)
    h5f.close()
    print("Dataset saved @ %s" % path)


def divide_segmentation(segmentation):
    layer1 = np.copy(segmentation)
    layer2 = np.copy(segmentation)
    background = np.copy(segmentation)

    layer1[layer1==2] = 1
    layer2[layer2==1] = 0
    layer2[layer2 > 0] = 1
    background[background==1] = 5
    background[background == 0] = 1
    background[background>1] = 0
    # np.concatenate((layer1, layer2, background), axis=-1)
    # return layer1, layer2, background
    # a = np.concatenate((layer1, layer2, background), axis=-1)
    # return a
    return layer1  # liver


data_dir = "/content/liver_image_dataset/"
train_dir = data_dir + "train/"
val_dir = data_dir + "val/"
test_dir = data_dir+"/test"
save_dir = "/content/"

print("Obtaining Training Data:")
train_set = create_dataset(data_dir,'train', crop=True)
write_dataset(train_set, save_dir + "train_data.h5")

print("Obtaining Validation Data:")
val_set = create_dataset(data_dir, 'val', crop=True)
write_dataset(val_set, save_dir + "val_data.h5")

print("Obtaining Test Data:")
test_set = create_dataset(data_dir,'test',crop=True)
write_dataset(test_set, save_dir + "test_data.h5")

In [3]:


f = h5py.File("/content/train_data.h5", 'r')
input_train = f['data'][...]
label_train = f['truth'][...]
f.close()


In [4]:

f = h5py.File("/content/val_data.h5", 'r')
input_val = f['data'][...]
label_val = f['truth'][...]
f.close()


In [5]:

f = h5py.File("/content/val_data.h5", 'r')
input_test = f['data'][...]
label_test = f['truth'][...]
f.close()

In [6]:

#basic vnet model

def vnet_implementation(img_height, img_width, img_depth, n_channels, num_classes):

  #Inputs
  #img_height,img_width = height and width of the image, img_depth = depth(or no. of slices in image), channels = 1 for 3D medical images
  inputs = Input((img_height, img_width, img_depth, n_channels))

  #Downsampling

  #input convolution
  inp_conv = Conv3D(16, (5,5,5), padding = 'same', kernel_initializer='he_normal')(inputs)
  inp_prelu = keras.layers.PReLU()(inp_conv)

  #step_down_1

  conv1 = Conv3D(16, (5,5,5), padding = 'same', kernel_initializer = 'he_normal')(inp_prelu)
  prelu1 = keras.layers.PReLU()(conv1)
  d1 = Conv3D(32, kernel_size = 2, strides = 2, padding = 'same')(prelu1)

  #step_down_2
  conv2 = Conv3D(32, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(d1)
  prelu2 = keras.layers.PReLU()(conv2)
  conv2 = Conv3D(32, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(prelu2)
  prelu2 = keras.layers.PReLU()(conv2)
  d2 = Conv3D(64, kernel_size = 2, strides = 2, padding = 'same')(prelu2)

  #bottleneck
  conv3 = Conv3D(64, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(d2)
  prelu3 = keras.layers.PReLU()(conv3)
  conv3 = Conv3D(64, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(prelu3)
  prelu3 = keras.layers.PReLU()(conv3)

  #step_up_1
  up1 = Conv3DTranspose(32, kernel_size = 2, strides = 2, padding = 'same', kernel_initializer= 'he_normal')(prelu3)
  up1 = concatenate([up1,prelu2])
  conv4 = Conv3D(32, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(up1)
  prelu4 = keras.layers.PReLU()(conv4)
  conv4 = Conv3D(32, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(prelu4)
  prelu4 = keras.layers.PReLU()(conv4)

  #step_up_2
  up2 = Conv3DTranspose(16, kernel_size = 2, strides = 2, padding = 'same', kernel_initializer= 'he_normal')(prelu4)
  up2 = concatenate([up2,prelu1])
  conv5 = Conv3D(16, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(up2)
  prelu5 = keras.layers.PReLU()(conv5)
  conv5 = Conv3D(16, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(prelu5)
#  prelu5 = keras.layers.PReLU()(conv5)

  #output layer
  outputs = Conv3D(1, (1, 1, 1), activation='softmax')(conv5)


  model = Model(inputs=[inputs], outputs=[outputs])
  #compile model outside of this function to make it flexible.
  model.summary()
  return model

In [7]:

metrics = ['accuracy']

LR = 0.0001
optim = keras.optimizers.Adam(LR)

In [8]:
model = vnet_implementation(128, 128, 64, 1,29)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 64,   │              0 │ -                      │
│                           │ 1)                     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d (Conv3D)           │ (None, 128, 128, 64,   │          2,016 │ input_layer[0][0]      │
│                           │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu (PReLU)           │ (None, 128, 128, 64,   │     16,777,216 │ conv3d[0][0]           │
│                           │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_1 (Conv3D)         │ (None, 128, 128, 64,   │         32,016 │ p_re_lu[0][0]          │
│                           │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_1 (PReLU)         │ (None, 128, 128, 64,   │     16,777,216 │ conv3d_1[0][0]         │
│                           │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_2 (Conv3D)         │ (None, 64, 64, 32, 32) │          4,128 │ p_re_lu_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_3 (Conv3D)         │ (None, 64, 64, 32, 32) │        128,032 │ conv3d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_2 (PReLU)         │ (None, 64, 64, 32, 32) │      4,194,304 │ conv3d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_4 (Conv3D)         │ (None, 64, 64, 32, 32) │        128,032 │ p_re_lu_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_3 (PReLU)         │ (None, 64, 64, 32, 32) │      4,194,304 │ conv3d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_5 (Conv3D)         │ (None, 32, 32, 16, 64) │         16,448 │ p_re_lu_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_6 (Conv3D)         │ (None, 32, 32, 16, 64) │        512,064 │ conv3d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_4 (PReLU)         │ (None, 32, 32, 16, 64) │      1,048,576 │ conv3d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_7 (Conv3D)         │ (None, 32, 32, 16, 64) │        512,064 │ p_re_lu_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_5 (PReLU)         │ (None, 32, 32, 16, 64) │      1,048,576 │ conv3d_7[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_transpose          │ (None, 64, 64, 32, 32) │         16,416 │ p_re_lu_5[0][0]        │
│ (Conv3DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concaten

 Total params: 71,041,457 (271.00 MB)

 Trainable params: 71,041,457 (271.00 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
print(model.input_shape)
print(model.output_shape)

(None, 128, 128, 64, 1)
(None, 128, 128, 64, 1)


In [10]:
model_dir = "/content/vnet_weights.weights.h5"
callbacks = list()
callbacks.append(ModelCheckpoint(model_dir, monitor='val_loss', save_weights_only=True, save_best_only=True))
callbacks.append(CSVLogger(model_dir + "training.log", append=True))
callbacks.append(ReduceLROnPlateau(factor=0.5, patience=2, verbose=1))
callbacks.append(EarlyStopping(patience=2))

In [11]:
x_train, y_train = input_train, label_train
x_val, y_val = input_val, label_val

In [12]:
model.compile(optimizer = Adam(learning_rate=1e-8), loss = 'binary_crossentropy', metrics = metrics)

model.fit(x_train, y_train,
          batch_size=1, epochs=2,
          validation_data=(x_val, y_val), callbacks=callbacks)

Epoch 1/2


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (1, 128, 128, 64, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 287s 153s/step - accuracy: 0.0744 - loss: 0.6382 - val_accuracy: 0.0404 - val_loss: 0.6236 - learning_rate: 1.0000e-08
Epoch 2/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 321s 155s/step - accuracy: 0.0785 - loss: 0.6367 - val_accuracy: 0.0404 - val_loss: 0.6236 - learning_rate: 1.0000e-08


In [13]:
model_json = model.to_json()


In [14]:
keras.models.save_model(model, '/content/model.hdf5')



In [15]:
!pip install pyngrok


In [16]:
!pip install streamlit



In [17]:
#APP
%%writefile app.py
import streamlit as st
import numpy as np
import nibabel as nib
import h5py
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model, model_from_json

import scipy.misc
import scipy.ndimage

#path to model/train data/predictions
save_dir = "/content/"

# Load your model
model = load_model("/content/model.hdf5")

#weights for the model
weights_dir = save_dir+"vnet_weights." + "weights.h5"

#load weights to the model
model.load_weights(weights_dir)


#  load  images
def load_h5(file):
    h5_image = h5py.File(file,'r')
    data = h5_image.get('data')
    #truth = h5_image.get('truth')

    return data


def predict(test, model):
    predictions = []
    m = test.shape[0]
    st.write('Starting predictions:')
    st.write("0/%i (0%%)" % m)

    for i in range(m):
        image = test[i][np.newaxis, :, :, :]
        prediction = model.predict(image, steps=1)
        prediction = np.squeeze(prediction)
        predictions.append(prediction)
        st.write("%i/%i (%i%%)" % (i + 1, m, ((i + 1) / m * 100)))

    predictions = np.array(predictions)

    st.write('Predictions obtained with shape:')
    st.write(predictions.shape)
    return predictions


def write_predictions(predicitons, path):
    h5f = h5py.File(path, 'w')
    h5f.create_dataset(name='predictions', data=predicitons)
    h5f.close()
    st.write('Predictions saved to ' + path)


# app
st.title('3D Medical Image Segmentation')

uploaded_h5_file = st.file_uploader('Choose a  hd5 file', type = 'h5')

if uploaded_h5_file is not None:
    x_test = load_h5(uploaded_h5_file)
else:
    st.write('No file uploaded yet')

save_path = "C:/Users/dell/Downloads/"

if st.button('Predict Segmentation'):
    predictions = predict(x_test, model)
    predictions = np.array(np.split(predictions, x_test.shape[0], axis=0))
    write_predictions(predictions, save_dir+'predictions_vnet_from_app.h5')

def download_file(file_path):
    with open(file_path, "rb") as file:
        btn = st.download_button(
            label="Download Predictions",
            data=file,
            file_name="predictions_vnet_from_app.h5",
            mime="application/octet-stream"
        )
download_file('/content/predictions_vnet_from_app.h5')


Overwriting app.py


In [18]:
import os
os.system('streamlit run app.py &')

0

In [19]:
!ngrok authtoken 2k5pt2lLu8HQmVXFol85HtzvPiz_2mhdcrH7MdFemnLzkQGyJ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [20]:
from pyngrok import ngrok


# Set up a new ngrok tunnel to the Streamlit app
public_url = ngrok.connect(8501)
print(f'Public URL: {public_url}')


Public URL: NgrokTunnel: "https://774e-35-224-74-216.ngrok-free.app" -> "http://localhost:8501"
